In [ ]:
reload_lamb()

# Tutorial: adding composition operations
### Author: Kyle Rawlins

You can write a new composition rule in three ways, in order of complexity (from simplest to hardest):

1. Write a metalanguage combinator that describes the rule and instantiate it as a composition rule.  This involves variants of the following syntax:
        system.add_binary_rule(combinator, "NAME")
2. Write a python function that describes the rule (as an operation over meta-language elements) and instantiate it as a composition rule.  The above `add_rule` call expects curried functions, and python functions are usually uncurried, but the call is essentially the same:
       system.add_binary_rule_uncurried(function, "names")
3. Write a python function that operates on object language elements (instances of `lang.SingletonComposable`) and produces an object language element (typically an instance of `lang.Composite`).

Where possible, you should apply strategy 1, and then 2.  Both of these handle a whole bunch of bookkeeping for you.  Doing 3 correctly in the general case often requires some detailed knowledge of the internals of the lambda notebook, and it is easy to make mistakes with the most obvious implementations.  (Sometimes it is unavoidable, as in the monster example below, or Predicate Abstraction.)

### Example 1.  A unary operator: existential closure

The simplest kind of operation to add is a unary operation, and existential closure is a good example of this.  (See the type shifting tutorial notebook for more examples of unary operations.)  The idea of existential closure is to existentially bind of some variable that is currently exposed as a lambda term in a formula.  The idea is easiest to see in the form of a combinator.  Existential closure is usually introduced at other types, but I will just use `e` for exemplification:

In [ ]:
ec_combinator = te("lambda f_<e,t> : Exists x_e : f(x)")
ec_combinator

Any metalanguage function is also a python function, so you can do stuff like the following directly to see how this works:

In [ ]:
%lamb ||cat|| = L x_e : Cat(x)
%lamb ||gray|| = L x_e : Gray(x)
ec_combinator(cat.content).reduce_all()

Adding this to the composition system is straightforward.  The following code makes a copy of one of the default systems and adds the existential closure rule as a unary rule based on the combinator:

In [ ]:
system = lang.td_system.copy()
system.add_unary_rule(ec_combinator, "EC")
lang.set_system(system)
system

In [ ]:
cat * gray

One way to force a unary rule to apply is to use `* None`:

In [ ]:
ectest = (cat * gray) * None
ectest

In [ ]:
ectest.tree(derivations=True)

In [ ]:
ectest.trace()

### Appendix 1: other ways of introducing existential closure

Behind the scenes, `add_unary_rule` constructs a CompositionOp using a factory function.  We could unpack this as follows:

In [ ]:
ec_rule = lang.unary_factory(ec_combinator, "EC", typeshift=False)
system = lang.td_system.copy()
system.add_rule(ec_rule)
lang.set_system(system)
%lamb ||cat|| = L x_e : Cat(x)
%lamb ||gray|| = L x_e : Gray(x)
system

In [ ]:
cat * None

A more elaborate thing would be to try strategy 3 above: write a function that manipulates object language elements to perform existential closure.  There is a bunch of extra complexity that you need to keep track of when doing this, including metalanguage assignments.  The following is an example function that implements existential closure still using the combinator internally; one could of course build the result in some other way (but this is surprisingly easy to get wrong).  Note that the type checking isn't strictly necessary, as it would happen from the combinator -- it is here for example purposes.

In [ ]:
def ec_fun_direct(f, assignment=None):
    ts = meta.get_type_system() # load the current type system
    if not (ts.eq_check(f.type, types.type_property)): # is the type a property type?  (<e,t>)
        raise types.TypeMismatch(f, None, "Existential closure for <e,t>")  # if not, raise an exception.
    ec_expr = te("lambda f_<e,t> : Exists x_e : f(x)") # use the combinator
    result = ec_expr.apply(f.content.under_assignment(assignment)).reduce() # reduce the outermost lambda
    return lang.UnaryComposite(f, result, source="EC-direct(%s)" % (f.name))

ec_fun_direct(cat).content

To use this you need to build a CompositionOp out of it.  Once you have this, add it to the composition system as usual

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.UnaryCompositionOp("EC-direct", ec_fun_direct, typeshift=False, reduce=True))
lang.set_system(system)
system

In [ ]:
cat * None

### Example 2.  A binary rule: Predicate Modification

What about binary composition operations?  Predicate modification comes built in, but it is useful to see how one might construct PM.  Like the built-in version, this is restricted to type `<e,t>`.

In [ ]:
pm_op = %te L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)
pm_op

In [ ]:
pm_op(cat.content)(gray.content).reduce_all().derivation

Now, Predicate Modification can be constructed directly using this combinator.

In [ ]:
system = lang.td_system.copy()
system.remove_rule("PM")
system.add_binary_rule(pm_op, "PM2")
lang.set_system(system)
%lamb ||cat|| = L x_e : Cat(x)
%lamb ||gray|| = L x_e : Gray(x)
system

In [ ]:
(cat * gray).tree()

See the neo-Davidsonian event semantics fragment for an example of how to write a generalized PM that works for polymorphic types.

### Appendix 2: an object-language implementation for Predicate Modification

Once again, you could write a python function that does the work.  This one still uses the combinator internally.
  * Tangent: your instinct may be to construct the result directly by building up the right TypedExpression.  This is certainly possible, but it is surprisingly tricky to get right; I encourage you to find solutions that involve combinators.
  * One reason is that you would have to deal with some issues in variable renaming to handle the general case of this; using beta reduction via a combinator ensures that this is all taken care of.

In [ ]:
pm_op = lang.te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")

def pm_fun(fun1, fun2, assignment=None):
    """H&K predicate modification -- restricted to type <e,t>."""
    ts = meta.get_type_system()
    if not (ts.eq_check(fun1.type, types.type_property) and 
            ts.eq_check(fun2.type, types.type_property)):
        raise TypeMismatch(fun1, fun2, "Predicate Modification")
    c1 = fun1.content.under_assignment(assignment)
    c2 = fun2.content.under_assignment(assignment)
    result = pm_op.apply(c1).apply(c2).reduce_all()
    return lang.BinaryComposite(fun1, fun2, result)

### Example 3: Binding evaluation parameters

A second kind of unary operation involves abstracting over a free variable in the metalanguage expression.  This can be thought of as "monstrous" shifting in the sense of Kaplan.  The following code sketches an implementation of this.

In [ ]:
reload_lamb()

In [ ]:
system = lang.td_system.copy()
speaker = lang.te("speaker_e")
system.assign_controller = lang.AssignmentController(specials=[speaker])
lang.set_system(system)
%lamb ||cat|| = L x_e : Cat(x)
%lamb ||gray|| = L x_e : Gray(x)

In [ ]:
i = lang.Item("I", lang.te("speaker_e"))
i

The following function binds instances of the variable `speaker` to a lambda term.  Notice that this function doesn't do anything to ensure that there are no free instances of `x` in `f`; this is one of the complexities you may have to deal with when implementing non-combinator-based composition operations.  However, this is basically a reasonable assumption for a case like this under normal practice.

In [ ]:
def monster_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"speaker": lang.te("x_e")})
    result = meta.LFun(types.type_e, f.under_assignment(new_a), varname="x")
    return lang.UnaryComposite(f, result, source="M(%s)" % (f.name))

monster_fun(i)

In [ ]:
system.add_rule(lang.UnaryCompositionOp("Monster", monster_fun))

In [ ]:
m_test = i * None
m_test

In [ ]:
cat * i

In [ ]:
((cat * i) * None).trace()

In [ ]:
%lamb ||john|| = John_e
(john * cat) * None # results in vacuous binding